In [1]:
from __future__ import print_function

import tensorflow as tf
import pandas as pd
from sklearn import preprocessing

In [2]:
# Prepare data for training, validation and testing

TRAIN_FILE = 'D:\Study\Ostfold\MachineLearning\git\data\ohenc_data_colNames.train';
VAL_FILE = 'D:\Study\Ostfold\MachineLearning\git\data\ohenc_data_colNames.val';
TEST_FILE = 'D:\Study\Ostfold\MachineLearning\git\data\ohenc_data_colNames.test';

# use one of 2 labels
redundant_label = 'outcome<50K'
label_name = 'outcome>50K'

# training data
train = pd.read_table(TRAIN_FILE, sep=' ')
train.pop(redundant_label)
train_x, train_y = train, train.pop(label_name)

# validation data
val = pd.read_table(VAL_FILE, sep=' ')
val.pop(redundant_label)
val_x, val_y = val, val.pop(label_name)

# testing data
test = pd.read_table(TEST_FILE, sep=' ')
test.pop(redundant_label)
test_x, test_y = test, test.pop(label_name)

display(train_x.head())
display(train_y.head())

sample_train_x = train_x.iloc[1:20, :]
sample_train_y = train_y.iloc[1:20]

sample_train_x.head()

,age,workclassMissing,workclassFederal-gov,workclassLocal-gov,workclassNever-worked,workclassPrivate,workclassSelf-emp-inc,workclassSelf-emp-not-inc,workclassState-gov,workclassWithout-pay,...,native-countryPortugal,native-countryPuerto-Rico,native-countryScotland,native-countrySouth,native-countryTaiwan,native-countryThailand,native-countryTrinadad-Tobago,native-countryUnited-States,native-countryVietnam,native-countryYugoslavia
0,-1.286609,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.395073,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.029490,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,-1.286609,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0.833773,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


0    0
1    0
2    0
3    0
4    1
Name: outcome>50K, dtype: int64

,age,workclassMissing,workclassFederal-gov,workclassLocal-gov,workclassNever-worked,workclassPrivate,workclassSelf-emp-inc,workclassSelf-emp-not-inc,workclassState-gov,workclassWithout-pay,...,native-countryPortugal,native-countryPuerto-Rico,native-countryScotland,native-countrySouth,native-countryTaiwan,native-countryThailand,native-countryTrinadad-Tobago,native-countryUnited-States,native-countryVietnam,native-countryYugoslavia
1,0.395073,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.029490,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,-1.286609,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0.833773,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
5,-0.043626,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [3]:
# Parameters
learning_rate = 0.0001
num_steps = 1500
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 1024 # 1st layer number of neurons
n_hidden_2 = 1024 # 2nd layer number of neurons
n_hidden_3 = 512
n_hidden_4 = 256
num_input = 108 # data input 
num_classes = 2 #  total classes (0 means outcome<=50)

In [4]:
# Define the input function for training
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(100).repeat(1000).batch(batch_size)

    # Return the dataset.
    return dataset.make_one_shot_iterator().get_next()

In [5]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset.make_one_shot_iterator().get_next()

In [6]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))


In [8]:
from numpy import random
tf.logging.set_verbosity(tf.logging.FATAL)
max_count = 100
hidden_set = [2048, 1024, 512, 256, 128, 64, 32, 16] 
for count in range(max_count):
    lr = 10**random.uniform(-2, -6)
    dropout = 10**random.uniform(-1,0)
    layers = random.randint(1, 4)
    hidden_units = random.randint(1, size=layers)
    for i in range(layers):
        hidden_units[i] = hidden_set[random.randint(0,8)]
        
    hidden_units = sorted(hidden_units, reverse=True)
    
    classifier1 = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        hidden_units=hidden_units,
        n_classes=2,
        optimizer=tf.train.GradientDescentOptimizer(learning_rate=lr),
        dropout=dropout,
        config=tf.estimator.RunConfig(tf_random_seed=1234),
        model_dir="D:\Study\Ostfold\MachineLearning\\tmp\\abcdfdf"+str(count)
        )

    # Train the Model
    classifier1.train(lambda:train_input_fn(train_x, train_y, 128), steps=1000)
    # Evaluate the model.
    eval_result = classifier1.evaluate(input_fn=lambda:eval_input_fn(val_x, val_y, 128))
    print('({}) lr: {}, dropout: {}, hidden_units {}'.format(count, lr, dropout, hidden_units))
    print('val accuracy: {accuracy:0.3f}'.format(**eval_result))
    eval_result = classifier1.evaluate(input_fn=lambda:eval_input_fn(train_x, train_y, 128))
    print('train accuracy: {accuracy:0.3f}'.format(**eval_result))

(0) lr: 0.001990457470557965, dropout: 0.29719275365527476, hidden_units [2048, 128, 32]
val accuracy: 0.850
train accuracy: 0.863
(1) lr: 0.00010222517825896352, dropout: 0.4741728164660085, hidden_units [2048, 512]
val accuracy: 0.834
train accuracy: 0.846
(2) lr: 1.278903200569297e-05, dropout: 0.24911368787661944, hidden_units [512, 128]
val accuracy: 0.761
train accuracy: 0.766
(3) lr: 6.632672407105759e-06, dropout: 0.19944086972966854, hidden_units [1024, 512]
val accuracy: 0.757
train accuracy: 0.760
(4) lr: 1.77923899714402e-05, dropout: 0.2697454256706041, hidden_units [1024, 256, 64]
val accuracy: 0.773
train accuracy: 0.778
(5) lr: 9.300109734081148e-06, dropout: 0.18382548899444456, hidden_units [1024, 32, 16]
val accuracy: 0.757
train accuracy: 0.760
(6) lr: 0.0033732249201003667, dropout: 0.4748672497242498, hidden_units [1024, 64]
val accuracy: 0.849
train accuracy: 0.862
(7) lr: 5.67091847368546e-06, dropout: 0.27753275581915726, hidden_units [64]
val accuracy: 0.761
t

train accuracy: 0.854
(65) lr: 0.001084129066245104, dropout: 0.17506800532662511, hidden_units [1024]
val accuracy: 0.849
train accuracy: 0.859
(66) lr: 5.7117568690762886e-05, dropout: 0.8329075590896395, hidden_units [2048, 1024, 32]
val accuracy: 0.757
train accuracy: 0.760
(67) lr: 8.316605700292158e-06, dropout: 0.417750427444755, hidden_units [2048, 512, 256]
val accuracy: 0.757
train accuracy: 0.760
(68) lr: 3.073014498169213e-06, dropout: 0.11466545842592822, hidden_units [256]
val accuracy: 0.755
train accuracy: 0.759
(69) lr: 0.0003555482161717297, dropout: 0.19253019044726588, hidden_units [32]
val accuracy: 0.840
train accuracy: 0.853
(70) lr: 1.761707417835988e-05, dropout: 0.1556582032526734, hidden_units [2048, 1024, 128]
val accuracy: 0.810
train accuracy: 0.816
(71) lr: 0.0005397929158562112, dropout: 0.43600364815104836, hidden_units [64, 32]
val accuracy: 0.842
train accuracy: 0.854
(72) lr: 3.476528406842721e-06, dropout: 0.9586929993076423, hidden_units [2048, 64]

In [9]:
for count in range(50):
    lr = 10**random.uniform(-2, -4)
    dropout = random.uniform(0.1, 0.7)

    
    # Overfit small sample dataset
    classifier1 = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        hidden_units=[1024, 256, 16],
        n_classes=2,
        optimizer=tf.train.GradientDescentOptimizer(learning_rate=lr),
        dropout=dropout,
        config=tf.estimator.RunConfig(tf_random_seed=1234),
        model_dir="D:\Study\Ostfold\MachineLearning\\tmp\\abcdeffdfdfhxf"+str(count)
        )

    # Train the Model
    classifier1.train(lambda:train_input_fn(train_x, train_y, 128), steps=3000)
    # Evaluate the model.
    eval_result = classifier1.evaluate(input_fn=lambda:eval_input_fn(val_x, val_y, 128))
    print('({}) lr: {}, dropout: {} [512, 512, 64],'.format(count, lr, dropout))
    print('val accuracy: {accuracy:0.3f}'.format(**eval_result))
    eval_result = classifier1.evaluate(input_fn=lambda:eval_input_fn(train_x, train_y, 128))
    print('train accuracy: {accuracy:0.3f}'.format(**eval_result))

(0) lr: 0.00010875541632157389, dropout: 0.22445044240865972 [512, 512, 64],
val accuracy: 0.848
train accuracy: 0.859
(1) lr: 0.00020477308790879122, dropout: 0.5396498321333216 [512, 512, 64],
val accuracy: 0.846
train accuracy: 0.857
(2) lr: 0.0017793482778325346, dropout: 0.5005896061098153 [512, 512, 64],
val accuracy: 0.849
train accuracy: 0.864
(3) lr: 0.00047445620316303213, dropout: 0.17335924815263387 [512, 512, 64],
val accuracy: 0.852
train accuracy: 0.867
(4) lr: 0.0037110576073500485, dropout: 0.5257399684592727 [512, 512, 64],
val accuracy: 0.850
train accuracy: 0.864
(5) lr: 0.002035199949116932, dropout: 0.13017370839177486 [512, 512, 64],
val accuracy: 0.849
train accuracy: 0.871
(6) lr: 0.00043795423976567276, dropout: 0.34102370478233707 [512, 512, 64],
val accuracy: 0.848
train accuracy: 0.863
(7) lr: 0.00430636200930064, dropout: 0.41097507951848844 [512, 512, 64],
val accuracy: 0.851
train accuracy: 0.869
(8) lr: 0.0013005568551944665, dropout: 0.4564724634072958

In [14]:
lr = 0.0025
dropout = 0.3508
count = 0

classifier1 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[1024, 256, 16],
    n_classes=2,
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=lr),
    dropout=dropout,
    config=tf.estimator.RunConfig(tf_random_seed=1234),
    model_dir="D:\Study\Ostfold\MachineLearning\\tmp\\abcwerf"+str(count)
    )

# Train the Model
classifier1.train(lambda:train_input_fn(train_x, train_y, 128), steps=3000)
# Evaluate the model.
eval_result = classifier1.evaluate(input_fn=lambda:eval_input_fn(val_x, val_y, 128))
print('({}) lr: {}, dropout: {} [1024, 256, 16],'.format(count, lr, dropout))
print('val accuracy: {accuracy:0.3f}'.format(**eval_result))
eval_result = classifier1.evaluate(input_fn=lambda:eval_input_fn(train_x, train_y, 128))
print('train accuracy: {accuracy:0.3f}'.format(**eval_result))

INFO:tensorflow:Using config: {'_task_id': 0, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_session_config': None, '_save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002AA53558F60>, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_master': '', '_service': None, '_model_dir': 'D:\\Study\\Ostfold\\MachineLearning\\tmp\\abcwerf0', '_tf_random_seed': 1234}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into D:\Study\Ostfold\MachineLearning\tmp\abcwerf0\model.ckpt.
INFO:tensorflow:loss = 91.2946, step = 1
INFO:tensorflow:global_step/sec: 42.0306
INFO:tensorflow:loss = 42.338905, step = 101 (2.381 sec)
INFO:tensorflow:global_step/sec: 49.5117
INFO:tensorflow:loss = 39.397545, step = 201 (2.022 sec)
INFO:tensorflow:global_step/sec: 49.

In [15]:
eval_result = classifier1.evaluate(input_fn=lambda:eval_input_fn(test_x, test_y, 128))
print('test accuracy: {accuracy:0.3f}'.format(**eval_result))

INFO:tensorflow:Starting evaluation at 2018-03-09-15:04:50
INFO:tensorflow:Restoring parameters from D:\Study\Ostfold\MachineLearning\tmp\abcwerf0\model.ckpt-3000
INFO:tensorflow:Finished evaluation at 2018-03-09-15:04:54
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.85756403, accuracy_baseline = 0.76377374, auc = 0.9104009, auc_precision_recall = 0.7783076, average_loss = 0.3129437, global_step = 3000, label/mean = 0.23622628, loss = 39.80497, prediction/mean = 0.21572933
test accuracy: 0.858


In [46]:
# Overfit small sample dataset
classifier1 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[512, 512, 64],
    n_classes=2,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0),
    dropout=dropout,
    config=tf.estimator.RunConfig(tf_random_seed=1234),
    model_dir="D:\Study\Ostfold\MachineLearning\\tmp\\abc"+str(count)
    )

# train_spec = tf.estimator.TrainSpec(lambda:train_input_fn(sample_train_x, sample_train_y, 2), max_steps=200)
# eval_spec = tf.estimator.EvalSpec(input_fn=lambda:eval_input_fn(sample_train_x, sample_train_y, 2), start_delay_secs=1,
#     throttle_secs=7)
# tf.estimator.train_and_evaluate(classifier1, train_spec, eval_spec)

# # Train the Model
# classifier1.train(lambda:train_input_fn(sample_train_x, sample_train_y, 2), steps=200)
# # Evaluate the model.
# eval_result = classifier1.evaluate(
#     input_fn=lambda:eval_input_fn(sample_train_x, sample_train_y, 2))
# print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_save_summary_steps': 100, '_is_chief': True, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000027902313C50>, '_task_type': 'worker', '_task_id': 0, '_save_checkpoints_secs': 600, '_session_config': None, '_master': '', '_log_step_count_steps': 100, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmpozik6lvy', '_num_worker_replicas': 1, '_tf_random_seed': None}


In [49]:
# Train the Model
classifier1.train(lambda:train_input_fn(train_x, train_y, 128), steps=20000)
# Evaluate the model.
eval_result = classifier1.evaluate(
    input_fn=lambda:eval_input_fn(val_x, val_y, 128))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\Local\Temp\tmpozik6lvy\model.ckpt-20000
INFO:tensorflow:Saving checkpoints for 20001 into C:\Users\ADMIN\AppData\Local\Temp\tmpozik6lvy\model.ckpt.
INFO:tensorflow:loss = 40.916393, step = 20001
INFO:tensorflow:global_step/sec: 37.281
INFO:tensorflow:loss = 28.861237, step = 20101 (2.696 sec)
INFO:tensorflow:global_step/sec: 44.9716
INFO:tensorflow:loss = 30.419363, step = 20201 (2.231 sec)
INFO:tensorflow:global_step/sec: 43.7624
INFO:tensorflow:loss = 35.70481, step = 20301 (2.284 sec)
INFO:tensorflow:global_step/sec: 38.3031
INFO:tensorflow:loss = 35.54525, step = 20401 (2.598 sec)
INFO:tensorflow:global_step/sec: 44.3935
INFO:tensorflow:loss = 29.435596, step = 20501 (2.258 sec)
INFO:tensorflow:global_step/sec: 44.6095
INFO:tensorflow:loss = 19.887547, step = 20601 (2.246 sec)
INFO:tensorflow:global_step/sec: 44.5284
INFO:tensorflow:loss = 32.48165, step = 20701 (2.240 sec)


INFO:tensorflow:loss = 26.149574, step = 28001 (2.562 sec)
INFO:tensorflow:global_step/sec: 42.246
INFO:tensorflow:loss = 33.12738, step = 28101 (2.372 sec)
INFO:tensorflow:global_step/sec: 43.812
INFO:tensorflow:loss = 28.434061, step = 28201 (2.287 sec)
INFO:tensorflow:global_step/sec: 42.3981
INFO:tensorflow:loss = 28.788351, step = 28301 (2.348 sec)
INFO:tensorflow:global_step/sec: 44.1647
INFO:tensorflow:loss = 25.583525, step = 28401 (2.276 sec)
INFO:tensorflow:global_step/sec: 42.6346
INFO:tensorflow:loss = 20.619926, step = 28501 (2.340 sec)
INFO:tensorflow:global_step/sec: 42.6448
INFO:tensorflow:loss = 24.62146, step = 28601 (2.344 sec)
INFO:tensorflow:global_step/sec: 43.9665
INFO:tensorflow:loss = 26.437603, step = 28701 (2.271 sec)
INFO:tensorflow:global_step/sec: 44.7931
INFO:tensorflow:loss = 25.048466, step = 28801 (2.237 sec)
INFO:tensorflow:global_step/sec: 44.5182
INFO:tensorflow:loss = 29.285044, step = 28901 (2.241 sec)
INFO:tensorflow:global_step/sec: 40.572
INFO:

INFO:tensorflow:global_step/sec: 44.6017
INFO:tensorflow:loss = 27.370323, step = 36301 (2.225 sec)
INFO:tensorflow:global_step/sec: 45.3668
INFO:tensorflow:loss = 24.128872, step = 36401 (2.215 sec)
INFO:tensorflow:global_step/sec: 43.6764
INFO:tensorflow:loss = 17.242332, step = 36501 (2.294 sec)
INFO:tensorflow:global_step/sec: 44.7518
INFO:tensorflow:loss = 22.053783, step = 36601 (2.228 sec)
INFO:tensorflow:global_step/sec: 44.7483
INFO:tensorflow:loss = 23.91792, step = 36701 (2.235 sec)
INFO:tensorflow:global_step/sec: 44.7252
INFO:tensorflow:loss = 20.896883, step = 36801 (2.228 sec)
INFO:tensorflow:global_step/sec: 45.0261
INFO:tensorflow:loss = 23.646158, step = 36901 (2.230 sec)
INFO:tensorflow:global_step/sec: 45.0137
INFO:tensorflow:loss = 29.813223, step = 37001 (2.225 sec)
INFO:tensorflow:global_step/sec: 45.2377
INFO:tensorflow:loss = 30.07884, step = 37101 (2.219 sec)
INFO:tensorflow:global_step/sec: 45.1563
INFO:tensorflow:loss = 25.091843, step = 37201 (2.210 sec)
IN

In [44]:
train_spec = tf.estimator.TrainSpec(lambda:train_input_fn(train_x, train_y, 128), max_steps=20000)
eval_spec = tf.estimator.EvalSpec(input_fn=lambda:eval_input_fn(val_x, val_y, 128), start_delay_secs=1,
    throttle_secs=60)
tf.estimator.train_and_evaluate(classifier1, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 60 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmput_u2whs\model.ckpt.
INFO:tensorflow:loss = 86.694275, step = 1
INFO:tensorflow:global_step/sec: 21.8711
INFO:tensorflow:loss = 36.113388, step = 101 (4.585 sec)
INFO:tensorflow:global_step/sec: 25.6417
INFO:tensorflow:loss = 41.716995, step = 201 (3.903 sec)
INFO:tensorflow:global_step/sec: 25.6812
INFO:tensorflow:loss = 29.383612, step = 301 (3.878 sec)
INFO:tensorflow:global_step/sec: 25.8917
INFO:tensorflow:loss = 39.33056, step = 401 (3.872 sec)
INFO:tensorflow:global_step/sec: 25.7753
INFO:tensorflow:loss = 46.783184, step = 501 (3.883 sec)
INFO:tensorflow:global_step/sec: 25.7078
INFO:tensorflow:loss = 35.764126, step = 601 (3.896 sec)
INFO:tenso

INFO:tensorflow:Saving dict for global step 3104: accuracy = 0.85076004, accuracy_baseline = 0.75664055, auc = 0.9072543, auc_precision_recall = 0.7662389, average_loss = 0.3230547, global_step = 3104, label/mean = 0.24335943, loss = 41.255985, prediction/mean = 0.25426924
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\Local\Temp\tmput_u2whs\model.ckpt-3104
INFO:tensorflow:Saving checkpoints for 3105 into C:\Users\ADMIN\AppData\Local\Temp\tmput_u2whs\model.ckpt.
INFO:tensorflow:loss = 39.368546, step = 3105
INFO:tensorflow:global_step/sec: 21.8185
INFO:tensorflow:loss = 28.923882, step = 3205 (4.593 sec)
INFO:tensorflow:global_step/sec: 25.7517
INFO:tensorflow:loss = 39.223213, step = 3305 (3.884 sec)
INFO:tensorflow:global_step/sec: 25.3594
INFO:tensorflow:loss = 33.522537, step = 3405 (3.947 sec)
INFO:tensorflow:global_step/sec: 26.0778
INFO:tensorflow:loss = 42.86046, step = 3505 (3.832 sec)
INFO:tensorflow:global_step/se

INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Finished evaluation at 2018-03-02-16:43:09
INFO:tensorflow:Saving dict for global step 6358: accuracy = 0.8495317, accuracy_baseline = 0.75664055, auc = 0.9055677, auc_precision_recall = 0.76317596, average_loss = 0.3284554, global_step = 6358, label/mean = 0.24335943, loss = 41.945686, prediction/mean = 0.23569371
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\Local\Temp\tmput_u2whs\model.ckpt-6358
INFO:tensorflow:Saving checkpoints for 6359 into C:\Users\ADMIN\AppData\Local\Temp\tmput_u2whs\model.ckpt.
INFO:tensorflow:loss = 32.10576, step = 6359
INFO:tensorflow:global_step/sec: 22.9615
INFO:tensorflow:loss = 30.83982, step = 6459 (4.351 sec)
INFO:tensorflow:global_step/sec: 25.9637
INFO:tensorflow:loss = 26.34925, step = 6559 (3.

INFO:tensorflow:loss = 34.754303, step = 9588 (3.676 sec)
INFO:tensorflow:Saving checkpoints for 9660 into C:\Users\ADMIN\AppData\Local\Temp\tmput_u2whs\model.ckpt.
INFO:tensorflow:Loss for final step: 32.205605.
INFO:tensorflow:Starting evaluation at 2018-03-02-16:49:40
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\Local\Temp\tmput_u2whs\model.ckpt-9660
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Finished evaluation at 2018-03-02-16:49:45
INFO:tensorflow:Saving dict for global step 9660: accuracy = 0.84339017, accuracy_baseline = 0.75664055, auc = 0.9030009, auc_precision_recall = 0.7593345, average_loss = 0.33463755, global_step = 9660, label/mean = 0.24335943, loss = 42.735184, prediction/mean = 0.24415472
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\L

INFO:tensorflow:global_step/sec: 24.7609
INFO:tensorflow:loss = 27.073322, step = 12546 (4.047 sec)
INFO:tensorflow:global_step/sec: 25.2113
INFO:tensorflow:loss = 50.160736, step = 12646 (3.958 sec)
INFO:tensorflow:global_step/sec: 25.8067
INFO:tensorflow:loss = 30.198915, step = 12746 (3.876 sec)
INFO:tensorflow:Saving checkpoints for 12823 into C:\Users\ADMIN\AppData\Local\Temp\tmput_u2whs\model.ckpt.
INFO:tensorflow:Loss for final step: 37.36085.
INFO:tensorflow:Starting evaluation at 2018-03-02-16:56:23
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\Local\Temp\tmput_u2whs\model.ckpt-12823
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Finished evaluation at 2018-03-02-16:56:29
INFO:tensorflow:Saving dict for global step 12823: accuracy = 0.8338707, accuracy_baseline = 0.75664055, auc = 0.8979449, auc_precision_recall

Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.Session object at 0x000002792220B7B8>>
Traceback (most recent call last):
  File "c:\users\admin\anaconda2\envs\tensorflow\lib\site-packages\tensorflow\python\client\session.py", line 706, in __del__
    tf_session.TF_DeleteDeprecatedSession(self._session, status)
KeyboardInterrupt: 


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\Local\Temp\tmput_u2whs\model.ckpt-12823


KeyboardInterrupt: 

In [50]:
eval_result = classifier1.evaluate(
    input_fn=lambda:eval_input_fn(train_x, train_y, 128))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Starting evaluation at 2018-03-02-17:30:50
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\Local\Temp\tmpozik6lvy\model.ckpt-40000
INFO:tensorflow:Finished evaluation at 2018-03-02-17:30:56
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.9224892, accuracy_baseline = 0.75982803, auc = 0.9769112, auc_precision_recall = 0.9332975, average_loss = 0.17104793, global_step = 40000, label/mean = 0.24017198, loss = 21.840473, prediction/mean = 0.24023814

Test set accuracy: 0.922



In [8]:
count =1
lr = 0.00087
dropout = 0.0
classifier2 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[512, 512, 64],
    n_classes=2,
    optimizer=tf.train.AdamOptimizer(learning_rate=lr),
#     dropout=dropout,
    config=tf.estimator.RunConfig(tf_random_seed=1234),
    model_dir="D:\Study\Ostfold\MachineLearning\\tmp\\abcdeffhxfdfdfd"+str(count)
    )

# Train the Model
classifier2.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=3000)
# Evaluate the model.
eval_result = classifier2.evaluate(input_fn=lambda:eval_input_fn(val_x, val_y, 128))
print('({}) lr: {}, dropout: {} [512, 512, 64],'.format(count, lr, dropout))
print('val accuracy: {accuracy:0.3f}'.format(**eval_result))
eval_result = classifier2.evaluate(input_fn=lambda:eval_input_fn(val_x, val_y, 33000))
print('({}) lr: {}, dropout: {} [512, 512, 64],'.format(count, lr, dropout))
print('val accuracy: {accuracy:0.3f}'.format(**eval_result))
eval_result = classifier2.evaluate(input_fn=lambda:eval_input_fn(train_x, train_y, 128))
print('train accuracy: {accuracy:0.3f}'.format(**eval_result))

INFO:tensorflow:Using config: {'_task_type': 'worker', '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000277000A1B70>, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_is_chief': True, '_model_dir': 'D:\\Study\\Ostfold\\MachineLearning\\tmp\\abcdeffhxfdfdfd1', '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_service': None, '_num_worker_replicas': 1, '_task_id': 0, '_save_summary_steps': 100, '_master': '', '_tf_random_seed': 1234}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into D:\Study\Ostfold\MachineLearning\tmp\abcdeffhxfdfdfd1\model.ckpt.
INFO:tensorflow:loss = 92.9067, step = 1
INFO:tensorflow:global_step/sec: 30.6091
INFO:tensorflow:loss = 39.928703, step = 101 (3.277 sec)
INFO:tensorflow:global_step/sec: 43.3464
INFO:tensorflow:loss = 35.422104, step = 201 (2.301 sec)
INFO:tensorflow:glob

NameError: name 'dropout' is not defined

In [10]:
dropout = 0.0
count = 1
eval_result = classifier2.evaluate(input_fn=lambda:eval_input_fn(val_x, val_y, 128))
print('({}) lr: {}, dropout: {} [512, 512, 64],'.format(count, lr, dropout))
print('val accuracy: {accuracy:0.3f}'.format(**eval_result))
eval_result = classifier2.evaluate(input_fn=lambda:eval_input_fn(val_x, val_y, 33000))
print('({}) lr: {}, dropout: {} [512, 512, 64],'.format(count, lr, dropout))
print('val accuracy: {accuracy:0.3f}'.format(**eval_result))
eval_result = classifier2.evaluate(input_fn=lambda:eval_input_fn(train_x, train_y, 128))
print('train accuracy: {accuracy:0.3f}'.format(**eval_result))

INFO:tensorflow:Starting evaluation at 2018-03-07-16:50:35
INFO:tensorflow:Restoring parameters from D:\Study\Ostfold\MachineLearning\tmp\abcdeffhxfdfdfd1\model.ckpt-3000
INFO:tensorflow:Finished evaluation at 2018-03-07-16:50:39
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.8189774, accuracy_baseline = 0.75664055, auc = 0.85882926, auc_precision_recall = 0.6798087, average_loss = 0.59328187, global_step = 3000, label/mean = 0.24335943, loss = 75.76558, prediction/mean = 0.24598263
(1) lr: 0.00087, dropout: 0.0 [512, 512, 64],
val accuracy: 0.819
INFO:tensorflow:Starting evaluation at 2018-03-07-16:50:43
INFO:tensorflow:Restoring parameters from D:\Study\Ostfold\MachineLearning\tmp\abcdeffhxfdfdfd1\model.ckpt-3000
INFO:tensorflow:Finished evaluation at 2018-03-07-16:50:47
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.8189774, accuracy_baseline = 0.75664055, auc = 0.85882926, auc_precision_recall = 0.6798087, average_loss = 0.59328157, global_step = 300

In [96]:
# Build 2 hidden layer DNN with 1024, 1024, 512, 256 units respectively.
classifier4 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[1024, 1024],
    # The model must choose between 2 classes.
    n_classes=2,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001),
    dropout=0.5,
    )

# Train the Model
classifier4.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=num_steps)
# Evaluate the model.
eval_result = classifier4.evaluate(
    input_fn=lambda:eval_input_fn(val_x, val_y, batch_size))
print('\nValidation set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_session_config': None, '_task_type': 'worker', '_save_summary_steps': 100, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_save_checkpoints_steps': None, '_service': None, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmp8kj8eenz', '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000251A805ABE0>, '_is_chief': True, '_task_id': 0}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmp8kj8eenz\model.ckpt.
INFO:tensorflow:step = 1, loss = 2113.696
INFO:tensorflow:global_step/sec: 21.2262
INFO:tensorflow:step = 101, loss = 1655.4054 (4.720 sec)
INFO:tensorflow:global_step/sec: 23.7133
INFO:tensorflow:step = 201, loss = 1533.023 (4.216 se

In [91]:
# Build 2 hidden layer DNN with 1024, 1024, 512, 256 units respectively, dropout
classifier5 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[1024, 1024, 512, 256],
    # The model must choose between 2 classes.
    n_classes=2,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001),
    )

# Train the Model
classifier5.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=num_steps)
# Evaluate the model.
eval_result = classifier5.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, batch_size))
print('\nValidation set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_session_config': None, '_task_type': 'worker', '_save_summary_steps': 100, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_save_checkpoints_steps': None, '_service': None, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmpagdgs5le', '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002518E872EF0>, '_is_chief': True, '_task_id': 0}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmpagdgs5le\model.ckpt.
INFO:tensorflow:step = 1, loss = 1111.0056
INFO:tensorflow:global_step/sec: 18.4038
INFO:tensorflow:step = 101, loss = 137.97911 (5.427 sec)
INFO:tensorflow:global_step/sec: 20.1585
INFO:tensorflow:step = 201, loss = 112.16977 (4.970 

In [24]:
# Build 2 hidden layer DNN with 1024, 1024, 512, 256 units respectively, dropout
classifier6 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[1024, 256],
    # The model must choose between 2 classes.
    n_classes=2,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001),
    )

# Train the Model
classifier6.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=3000)
# Evaluate the model.
eval_result = classifier6.evaluate(
    input_fn=lambda:eval_input_fn(val_x, val_y, batch_size))
print('\nValidation set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_save_summary_steps': 100, '_is_chief': True, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002791A80EEB8>, '_task_type': 'worker', '_task_id': 0, '_save_checkpoints_secs': 600, '_session_config': None, '_master': '', '_log_step_count_steps': 100, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmpnqfn6ud2', '_num_worker_replicas': 1, '_tf_random_seed': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmpnqfn6ud2\model.ckpt.
INFO:tensorflow:loss = 93.46505, step = 1
INFO:tensorflow:global_step/sec: 43.9859
INFO:tensorflow:loss = 91.85483, step = 101 (2.292 sec)
INFO:tensorflow:global_step/sec: 52.4869
INFO:tensorflow:loss = 87.80405, step = 201 (1.904 sec

In [36]:
# Evaluate the model.
eval_result = classifier6.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, batch_size))
print('\nValidation set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Starting evaluation at 2018-03-01-12:37:20
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\Local\Temp\tmpqidjxw1c\model.ckpt-3000
INFO:tensorflow:Finished evaluation at 2018-03-01-12:37:24
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.848228, accuracy_baseline = 0.76377374, auc = 0.8976855, auc_precision_recall = 0.75689113, average_loss = 0.34015924, global_step = 3000, label/mean = 0.23622628, loss = 43.26666, prediction/mean = 0.23726822

Validation set accuracy: 0.848

